## code to simulate anwer 

In [18]:
# code1=""" import sympy as sp

# # Define variable
# x = sp.Symbol('x')

# # Example equation: 3x + 2 = 11
# equation = sp.Eq(3 * x + 2, 11)

# # Solve equation
# solution = sp.solve(equation, x)
# solution"""


# code3="""
# ```python
# import sympy as sp

# # Define variables
# x, y = sp.symbols('x y')

# # Example equations: 2x + y = 10, 3x - y = 5
# equations = [sp.Eq(2 * x + y, 10), sp.Eq(3 * x - y, 5)]

# # Solve the system of equations
# solution = sp.solve(equations, (x, y))
# solution
# ```
# """

# code4="""```python
# # Given values
# radius = 4  # in cm
# pi = 3.14

# # Calculate the area of the circle
# area = pi * (radius ** 2)
# print(area)
# ```"""

In [19]:
# code2 =''' 
# ```python
# import sympy as sp

# # Define variable
# x = sp.Symbol('x')

# # Example equation: x^2 - 5x + 6 = 0
# equation = sp.Eq(x**2 - 5 * x + 6, 0)

# # Solve equation
# solution = sp.solve(equation, x)
# solution
# ```
# '''

In [20]:
import os
import re
import signal
import subprocess
import tempfile
from collections import Counter
from contextlib import contextmanager
from dataclasses import dataclass

In [21]:



code1 = '''
```python
import sympy as sp

# Define variable
x = sp.Symbol('x')

# Example equation: 3x + 2 = 11
equation = sp.Eq(3 * x + 2, 11)

# Solve equation
solution = sp.solve(equation, x)
solution

```
'''


code2 = '''
```python
import sympy as sp

# Define variable
x = sp.Symbol('x')

# Example equation: x^2 - 5x + 6 = 0
equation = sp.Eq(x**2 - 5 * x + 6, 0)

# Solve equation
solution = sp.solve(equation, x)
solution
```
'''

code3 = '''
```python
import sympy as sp

# Example expression: (x^2 - 1) / (x - 1)
x = sp.Symbol('x')
expression = (x**2 - 1) / (x - 1)

# Simplify expression
simplified_expr = sp.simplify(expression)
simplified_expr
```
'''


code4 = '''
```python
import sympy as sp

# Define variables
x, y = sp.symbols('x y')

# Example equations: 2x + y = 10, 3x - y = 5
equations = [sp.Eq(2 * x + y, 10), sp.Eq(3 * x - y, 5)]

# Solve the system of equations
solution = sp.solve(equations, (x, y))
solution
```
'''



code5 = '''
```python
import sympy as sp

# Example numbers: 48 and 60
a = 48
b = 60

# Compute GCD
gcd = sp.gcd(a, b)
gcd
```
'''



code6 = '''
```python
import sympy as sp

# Example numbers: 12 and 18
a = 12
b = 18

# Compute LCM
lcm = sp.lcm(a, b)
lcm
```
'''



code7 = '''
```python
import sympy as sp

# Define variable
x = sp.Symbol('x')

# Example inequality: 2x - 3 > 5
inequality = sp.Gt(2 * x - 3, 5)

# Solve the inequality
solution = sp.solve(inequality, x)
solution
```
'''


code8 = '''
```python
import sympy as sp

# Define variable
x = sp.Symbol('x')

# Example expression: x^2 - 9
expression = x**2 - 9

# Factorize expression
factorized_expr = sp.factor(expression)
factorized_expr
```
'''


code9 = '''
```python
import sympy as sp

# Define variable
x = sp.Symbol('x')

# Example expression: (x + 2)^3
expression = (x + 2)**3

# Expand expression
expanded_expr = sp.expand(expression)
expanded_expr
```
'''


code10 = '''
```python
import sympy as sp

# Define variable
x = sp.Symbol('x')

# Example function: f(x) = 3x^2 + 4x + 1
function = 3 * x**2 + 4 * x + 1

# Compute the derivative
derivative = sp.diff(function, x)
derivative
```
'''


In [22]:
class PythonREPL:
    def __init__(self, timeout=5):
        self.timeout = timeout

    @contextmanager
    def time_limit(self, seconds):
        def signal_handler(*_):
            raise TimeoutError(f"Timed out after {seconds} seconds.")

        signal.signal(signal.SIGALRM, signal_handler)
        signal.alarm(seconds)
        try:
            yield
        finally:
            signal.alarm(0)

    def __call__(self, query):
        query = "import math\nimport numpy as np\nimport sympy as sp\n" + query
        query = query.strip().split("\n")
        if "print(" not in query[-1]:
            if "#" in query[-1]:
                query[-1] = query[-1].split("#")[0]
            query[-1] = "print(" + query[-1] + ")"
        query = "\n".join(query)
        with tempfile.TemporaryDirectory() as temp_dir:
            temp_file_path = os.path.join(temp_dir, "tmp.py")
            with open(temp_file_path, "w", encoding="utf-8") as f:
                f.write(query)
            with self.time_limit(self.timeout):
                result = subprocess.run(
                    ["python3", temp_file_path],
                    capture_output=True,
                    check=False,
                    text=True,
                    timeout=self.timeout,
                )
                if result.returncode == 0:
                    output = result.stdout
                    return True, output.strip()
                error_msg = result.stderr.strip()
                msgs = error_msg.split("\n")
                new_msgs = []
                want_next = False
                for m in msgs:
                    if "Traceback" in m:
                        new_msgs.append(m)
                    elif m == msgs[-1]:
                        new_msgs.append(m)
                    elif temp_file_path in m:
                        st = m.index('"/') + 1 if '"/' in m else 0
                        ed = m.index(temp_file_path) + 1 if temp_file_path in m else None
                        clr = m[st:ed] if not ed else m[st:]
                        m = m.replace(clr, "")
                        new_msgs.append(m)
                        want_next = True
                    elif want_next:
                        new_msgs.append(m)
                        want_next = False
                error_msg = "\n".join(new_msgs)
                return False, error_msg.strip()
            

def execute_completion(executor, completion, return_status, last_code_block):
    executions = re.findall(r"```python(.*?)```", completion, re.DOTALL)
    if len(executions) == 0:
        return completion, False if return_status else completion
    if last_code_block:
        executions = [executions[-1]]
    outputs = []
    successes = []
    for code in executions:
        success = False
        for lib in ("subprocess", "venv"):
            if lib in code:
                output = f"{lib} is not allowed"
                outputs.append(output)
                successes.append(success)
                continue
        try:
            success, output = executor(code)
        except TimeoutError as e:
            print("Code timed out")
            output = e
        if not success and not return_status:
            output = ""
        outputs.append(output)
        successes.append(success)
    output = str(outputs[-1]).strip()
    success = successes[-1]
    if return_status:
        return output, success
    return output


def postprocess_completion(text, return_status, last_code_block):
    executor = PythonREPL()
    result = execute_completion(executor, text, return_status=return_status, last_code_block=last_code_block)
    del executor
    return result

In [23]:
answer=postprocess_completion(code10,True,True)
print(answer)

('6*x + 4', True)


In [62]:
import re
import numpy as np

# Example function to define your executor or transformation logic
def replacement_function(num, modifier):
    return num + modifier  # Add the modifier to the number



class valuation:
    def __init__(self,code):
        self.sensitivity=None
        self.scale=None
        self.executor=PythonREPL()
        self.code=code
        self.simulates=None
        self.simualtes=[0,0.001,0.01,0.1,1,10,100,1000]
        
    def postprocess_completion(self,text, return_status=True, last_code_block=True):
        executor = PythonREPL()
        result = execute_completion(executor, text, return_status=return_status, last_code_block=last_code_block)
        del executor
        return result

    def modifier(self,code):
        pattern = r'\d+\.?\d*'
        modified_strings = []
        for modifier in self.simualtes:
 
            def replace_match(match):
                num = float(match.group())  # Convert the matched number to a float
                return str(replacement_function(num, modifier))  # Apply replacement with modifier

            modified_string = re.sub(pattern, replace_match,code)
            modified_strings.append(modified_string)
      
        return modified_strings
        
    def sestivity_check(self, actual_answer, predicted_answer):
        act_answers = []
        pre_answers = []

        # Process actual answers
        mods_act = self.modifier(actual_answer)
        for sti in mods_act:
            answer,sucess = self.postprocess_completion(sti)
            act_answers.append(answer)

        # Process predicted answers
        mods_pred = self.modifier(predicted_answer)
        for sti in mods_pred:
            answer,sucess = self.postprocess_completion(sti)
            pre_answers.append(answer)
        print(act_answers, pre_answers)
        # Convert answers to numpy arrays
        act_answers = (np.array(act_answers[1:], dtype=float)-np.full(7,act_answers[0],dtype=float))/np.array(self.simualtes[1:],dtype=float)
        pre_answers = (np.array(pre_answers[1:], dtype=float)-np.full(7,pre_answers[0],dtype=float))/np.array(self.simualtes[1:],dtype=float)

        print(act_answers, pre_answers)

        # Perform element-wise subtraction and division
        distance = act_answers - pre_answers
        print("distance",distance)
        distance = np.abs(distance)  

        return distance
        
    def constant_modifier(self):
        pass
            
      
        
    def type_check(self):
        pass
        
    
    
    def scale_check(self):
        pass
        

In [67]:
code20 = '''
```python
import sympy as sp

# Example numbers: 12 and 18
a = 12
b = 18

# Compute LCM
lcm = sp.lcm(a, b)
lcm=lcm*2
lcm
```
'''
code21='''
```python

import sympy as sp
length=12
width=
# Define symbolic variables for length and width
length, width = sp.symbols('length width')

# Area of the rectangle
area = length * width
```
'''




In [68]:
valuator=  valuation(code6)
valuator.sestivity_check(code6,code20)

['216.000000000000', '216.030001000000', '216.300100000000', '219.010000000000', '247.000000000000', '616.000000000000', '13216.0000000000', '1030216.00000000'] ['432.000000000000', '432.276032001000', '434.763201000000', '459.921000000000', '741.000000000000', '7392.00000000000', '1348032.00000000', '1032276432.00000']
[  30.001   30.01    30.1     31.      40.     130.    1030.   ] [2.76032001e+02 2.76320100e+02 2.79210000e+02 3.09000000e+02
 6.96000000e+02 1.34760000e+04 1.03227600e+06]
distance [-2.46031001e+02 -2.46310100e+02 -2.49110000e+02 -2.78000000e+02
 -6.56000000e+02 -1.33460000e+04 -1.03124600e+06]


array([2.46031001e+02, 2.46310100e+02, 2.49110000e+02, 2.78000000e+02,
       6.56000000e+02, 1.33460000e+04, 1.03124600e+06])

In [48]:
import re
import numpy as np

# Example function to define your executor or transformation logic
def replacement_function(num, modifier):
    return num + modifier  # Add the modifier to the number


class Valuation:
    def __init__(self, code):
        self.code = code
        self.executor = PythonREPL()
        self.simulates = [0.001, 0.01, 0.1, 1, 10, 100, 1000]  # Defined simulation factors
        
    def modifier(self, code):
        pattern = r'\d+\.?\d*'  # Regular expression to match numbers
        modified_strings = []
        
        for modifier in self.simulates:
            def replace_match(match):
                num = float(match.group())  # Convert the matched number to a float
                return str(replacement_function(num, modifier))  # Apply replacement with modifier

            modified_string = re.sub(pattern, replace_match, code)
            modified_strings.append(modified_string)
      
        return modified_strings
        
    def sensitivity_check(self, actual_answer, predicted_answer):
        act_answers = []
        pre_answers = []

        # Apply modifier to actual answer
        mods_act = self.modifier(actual_answer)
        print(f"Modified actual answer: {mods_act}")
        
        for sti in mods_act:
            answer = self.executor(sti)
            act_answers.append(float(answer))
        
        # Apply modifier to predicted answer
        mods_pred = self.modifier(predicted_answer)
        print(f"Modified predicted answer: {mods_pred}")

        for sti in mods_pred:
            answer = self.executor(sti)
            pre_answers.append(float(answer))
        
        # Convert answers to NumPy arrays
        act_answers = np.array(act_answers)
        pre_answers = np.array(pre_answers)
        
        print(f"Actual answers: {act_answers}, Predicted answers: {pre_answers}")
        
        # Calculate sensitivity (difference divided by simulates array)
        distance = (act_answers - pre_answers) / np.array(self.simulates)
        
        return distance

    def type_check(self):
        pass
        
    def scale_check(self):
        pass




In [27]:
codep="\n```python\nimport sympy as sp\n\n# Define variable\nx = sp.Symbol('x')\n\n# Example equation: x^2.0 - 5.0x + 6.0 = 0.0\nequation = sp.Eq(x**2.0 - 5.0 * x + 6.0, 0.0)\n\n# Solve equation\nsolution = sp.solve(equation, x)\nsolution\n```\n"
answer=postprocess_completion(codep,True,True)
print(answer)

('[2.00000000000000, 3.00000000000000]', True)


In [1]:
import ast

# Example code as a string
code = '''
import sympy as sp

# Example numbers: 48 and 60
a = 48
b = 60

# Compute GCD
gcd = sp.gcd(a, b)
gcd
'''

# Parse the code into an AST
tree = ast.parse(code)

# Pretty print the AST tree structure
print(ast.dump(tree, indent=4))


Module(
    body=[
        Import(
            names=[
                alias(name='sympy', asname='sp')]),
        Assign(
            targets=[
                Name(id='a', ctx=Store())],
            value=Constant(value=48)),
        Assign(
            targets=[
                Name(id='b', ctx=Store())],
            value=Constant(value=60)),
        Assign(
            targets=[
                Name(id='gcd', ctx=Store())],
            value=Call(
                func=Attribute(
                    value=Name(id='sp', ctx=Load()),
                    attr='gcd',
                    ctx=Load()),
                args=[
                    Name(id='a', ctx=Load()),
                    Name(id='b', ctx=Load())],
                keywords=[])),
        Expr(
            value=Name(id='gcd', ctx=Load()))],
    type_ignores=[])


In [2]:
pip install graphviz


Note: you may need to restart the kernel to use updated packages.


In [3]:
import ast
import graphviz
import matplotlib.pylot as plt
# Function to visualize the AST as a Graphviz diagram
def ast_to_graphviz(node, graph=None):
    if graph is None:
        graph = graphviz.Digraph()
    
    # Create a unique node identifier
    node_id = str(id(node))
    
    # Add current node to the graph
    graph.node(node_id, label=type(node).__name__)
    
    # Traverse child nodes and add them to the graph
    for field, value in ast.iter_fields(node):
        if isinstance(value, list):
            for item in value:
                if isinstance(item, ast.AST):
                    child_id = str(id(item))
                    graph.node(child_id, label=type(item).__name__)
                    graph.edge(node_id, child_id)
                    ast_to_graphviz(item, graph)  # Recursively add children
        elif isinstance(value, ast.AST):
            child_id = str(id(value))
            graph.node(child_id, label=type(value).__name__)
            graph.edge(node_id, child_id)
            ast_to_graphviz(value, graph)  # Recursively add children
    return graph

# Example Python code to be visualized as an AST
code= '''
# ```python
import sympy as sp

# Example numbers: 12 and 18
a = 12
b = 18

# Compute LCM
lcm = sp.lcm(a, b)
lcm
# ```
'''

# Parse the code into an AST
tree = ast.parse(code)

# Convert AST to Graphviz diagram
graph = ast_to_graphviz(tree)

# Save the graph to a file (e.g., ast_tree)
graph.render('ast_tree', format='png', cleanup=True)

# Optionally, display the graph if you have Jupyter or IPython
# graph.view(filename="/kaggle/working/ast_tree.png")
image=plt.imread("/kaggle/working/ast_tree.png")
plt.imshow(image)



ModuleNotFoundError: No module named 'matplotlib.pylot'

In [14]:
import ast

# Example Python code as a string
code = '''
import sympy as sp

a = 48
b = 60
gcd = sp.gcd(a, b)
gcd
'''

# Parse the code into an AST
tree = ast.parse(code)

# Function to recursively print AST nodes, including constant values
def print_ast_with_constants(node, indent=''):
    # Check if the node is a Constant, and print the value if it is
    if isinstance(node, ast.Constant):
        print(f"{indent}Constant: {node.value}")
#     else:
#         print(f"{indent}{type(node).__name__}")

    # Recursively print children of the current node
    for field, value in ast.iter_fields(node):
        if isinstance(value, list):
            for item in value:
                if isinstance(item, ast.AST):
                    print_ast_with_constants(item, indent + '    ')
        elif isinstance(value, ast.AST):
            print_ast_with_constants(value, inde nt + '    ')

# Print the AST with constants
print_ast_with_constants(tree)


        Constant: 48
        Constant: 60


In [17]:
import ast

# Example Python code as a string
code = '''
import sympy as sp

a = 48
b = 60
gcd = sp.gcd(a, b)
print(gcd)
'''

# Parse the code into an AST
tree = ast.parse(code)

# Function to recursively print and replace AST constant values
def replace_constants(node, new_value):
    # Check if the node is a Constant, and replace the value if it is
    if isinstance(node, ast.Constant):
        
        node.value =node.value+ new_value
        print(f"Replacing Constant: {node.value} ")

    # Recursively traverse and modify children of the current node
    for field, value in ast.iter_fields(node):
        if isinstance(value, list):
            for item in value:
                if isinstance(item, ast.AST):
                    replace_constants(item, new_value)
        elif isinstance(value, ast.AST):
            replace_constants(value, new_value)

# Replace constants in the AST with a new value (e.g., 100)
replace_constants(tree, 6)

# Compile the modified AST back into executable code
compiled_code = compile(tree, filename="<ast>", mode="exec")

# Execute the modified code
exec_namespace = {}

# Execute the modified code and store the variables in exec_namespace
exec(compiled_code, exec_namespace)

# Now you can access the variables defined in the code
gcd_value = exec_namespace.get('gcd', None)
a_value = exec_namespace.get('a', None)
b_value = exec_namespace.get('b', None)

print(f"a = {a_value}, b = {b_value}, gcd = {gcd_value}")


Replacing Constant: 54 
Replacing Constant: 66 
6
a = 54, b = 66, gcd = 6


## NOt useful now 

In [69]:
import sympy as sp
import numpy as np

# Function to compute derivative numerically
def numerical_derivative(code, var_name, points, h=1e-5):
    derivatives = []
    
    for point in points:
        # Create a local namespace for the exec function
        local_vars = {}
        
        # Modify the code to substitute the variable value
        code_at_point = code.replace(f'{var_name}=', f'{var_name}={point}')
        
        # Execute the code at the original point
        exec(code_at_point, {}, local_vars)
        f_x = local_vars.get('lcm', local_vars.get('area', None))
        
        # Execute the code at the point + h
        code_at_point_h = code.replace(f'{var_name}=', f'{var_name}={point + h}')
        exec(code_at_point_h, {}, local_vars)
        f_x_h = local_vars.get('lcm', local_vars.get('area', None))
        
        # Calculate the numerical derivative
        derivative = (f_x_h - f_x) / h
        derivatives.append(derivative)
    
    return derivatives

# Example usage
code20 = '''
import sympy as sp

# Example numbers: 12 and 18
a = 12
b = 18

# Compute LCM
lcm = sp.lcm(a, b)
lcm = lcm * 2
'''

code21 = '''
import sympy as sp

# Define symbolic variables for length and width
length = 12
width = 5  # Assuming a fixed width for the example

# Area of the rectangle
area = length * width
'''

# Points at which we want to find the derivative
points = [0, 0.001, 0.01, 0.1, 1, 10, 100, 1000]

# Calculate the numerical derivative for code21 (Area of the rectangle with respect to length)
derivatives_code21 = numerical_derivative(code21, 'length', points)

# Print results for code21
print("Derivatives for code21 (area of rectangle):")
for point, derivative in zip(points, derivatives_code21):
    print(f"Derivative at length = {point}: {derivative}")

# Calculate the numerical derivative for code20 (LCM computation with respect to 'a')
derivatives_code20 = numerical_derivative(code20, 'a', points)

# Print results for code20
print("\nDerivatives for code20 (LCM computation):")
for point, derivative in zip(points, derivatives_code20):
    print(f"Derivative at a = {point}: {derivative}")


Derivatives for code21 (area of rectangle):
Derivative at length = 0: 0.0
Derivative at length = 0.001: 0.0
Derivative at length = 0.01: 0.0
Derivative at length = 0.1: 0.0
Derivative at length = 1: 0.0
Derivative at length = 10: 0.0
Derivative at length = 100: 0.0
Derivative at length = 1000: 0.0

Derivatives for code20 (LCM computation):
Derivative at a = 0: 0
Derivative at a = 0.001: 0
Derivative at a = 0.01: 0
Derivative at a = 0.1: 0
Derivative at a = 1: 0
Derivative at a = 10: 0
Derivative at a = 100: 0
Derivative at a = 1000: 0
